<a href="https://colab.research.google.com/github/shivarkrish18/college-projects/blob/main/Diabetic_Retinopathy_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import os
import zipfile
from google.colab import drive
drive.mount('/content/gdrive')
print(tf.__version__)

          
          
          




Mounted at /content/gdrive
2.3.0


In [ ]:
!unzip -o "/content/gdrive/My Drive/TEST.zip"

Streaming output truncated to the last 5000 lines.
  inflating: dataset_test/level0/35922_left.jpeg  
  inflating: dataset_test/level0/35922_right.jpeg  
  inflating: dataset_test/level0/35925_left.jpeg  
  inflating: dataset_test/level0/35925_right.jpeg  
  inflating: dataset_test/level0/35938_left.jpeg  
  inflating: dataset_test/level0/35938_right.jpeg  
  inflating: dataset_test/level0/35941_left.jpeg  
  inflating: dataset_test/level0/35944_left.jpeg  
  inflating: dataset_test/level0/35944_right.jpeg  
  inflating: dataset_test/level0/35945_right.jpeg  
  inflating: dataset_test/level0/35947_left.jpeg  
  inflating: dataset_test/level0/35952_left.jpeg  
  inflating: dataset_test/level0/35952_right.jpeg  
  inflating: dataset_test/level0/35954_left.jpeg  
  inflating: dataset_test/level0/35954_right.jpeg  
  inflating: dataset_test/level0/35956_left.jpeg  
  inflating: dataset_test/level0/35956_right.jpeg  
  inflating: dataset_test/level0/35960_left.jpeg  
  inflating: dataset_te

In [ ]:
!unzip -o "/content/gdrive/My Drive/TRAIN.zip"

Streaming output truncated to the last 5000 lines.
 extracting: dataset_train/level0/34267_left.jpeg  
 extracting: dataset_train/level0/34267_right.jpeg  
 extracting: dataset_train/level0/34268_left.jpeg  
 extracting: dataset_train/level0/34268_right.jpeg  
 extracting: dataset_train/level0/34269_left.jpeg  
 extracting: dataset_train/level0/34269_right.jpeg  
 extracting: dataset_train/level0/34275_left.jpeg  
 extracting: dataset_train/level0/34275_right.jpeg  
 extracting: dataset_train/level0/34277_left.jpeg  
 extracting: dataset_train/level0/34277_right.jpeg  
 extracting: dataset_train/level0/34280_left.jpeg  
 extracting: dataset_train/level0/34280_right.jpeg  
 extracting: dataset_train/level0/34282_left.jpeg  
 extracting: dataset_train/level0/34283_left.jpeg  
 extracting: dataset_train/level0/34283_right.jpeg  
 extracting: dataset_train/level0/34287_left.jpeg  
 extracting: dataset_train/level0/34287_right.jpeg  
 extracting: dataset_train/level0/34288_right.jpeg  
 ext

In [ ]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense, Dropout
from keras.optimizers import SGD
classifier = Sequential()
classifier.add(Conv2D(128, (3, 3), input_shape = (256, 256, 3), activation = 'relu'))
classifier.add(Conv2D(128, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Dropout(0.5))
classifier.add(Flatten())
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dropout(0.5))
classifier.add(Dense(units = 5, activation = 'softmax'))
classifier.compile(optimizer = 'adadelta', loss = 'categorical_crossentropy', metrics = ['accuracy'])
print(classifier.summary())
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)
training_set = train_datagen.flow_from_directory('/content/dataset_train',
                                                 target_size = (256,256),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')
test_set = test_datagen.flow_from_directory('/content/dataset_test',
                                            target_size = (256, 256),
                                            batch_size = 32,
                                            class_mode = 'categorical')
history = classifier.fit_generator(training_set,
                         steps_per_epoch = 878,
                         epochs = 25,
                         validation_data = test_set,
                         validation_steps = 220
                         )
labels = (training_set.class_indices)
print(labels)
classifier.save("cnn_dr_model.h5")
classifier.save_weights('my_model_weights.h5',overwrite=True)



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 254, 254, 128)     3584      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 252, 252, 128)     147584    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 126, 126, 128)     0         
_________________________________________________________________
dropout (Dropout)            (None, 126, 126, 128)     0         
_________________________________________________________________
flatten (Flatten)            (None, 2032128)           0         
_________________________________________________________________
dense (Dense)                (None, 128)               260112512 
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0

In [ ]:
import numpy as np
import cv2
from PIL import Image
from keras.preprocessing import image
from keras.models import load_model
classifier = load_model('cnn_dr_model.h5')   
classifier.load_weights('my_model_weights.h5')
print(classifier.summary())
labels = {0:'level0',1:'level1',2:'level2',3:'level3',4:'level4'}
im=Image.open('37708_left.jpeg')
new=im.resize((256,256))
image=np.array(new)
gcImg = image[:,:,1]
curImg = gcImg
clahe = cv2.createCLAHE()
clImg = clahe.apply(curImg)
curImg = clImg
strEl = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(6,6))
dilateImg = cv2.dilate(curImg, strEl)
curImg = dilateImg
retValue, threshImg = cv2.threshold(curImg, 220, 255, cv2.THRESH_BINARY)
curImg = threshImg
medianImg = cv2.medianBlur(curImg,5)
curImg = medianImg
cv2.imwrite('ip.jpeg',curImg)
Img=cv2.imread('ip.jpeg')
       
test_image = np.expand_dims(Img, axis = 0)
test_image = test_image/255        
result = classifier.predict(test_image)
disease_detected = labels[np.argmax(result[0],axis=0)]
print("Level of Diabetic Retinopathy - ",disease_detected)
       

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 254, 254, 128)     3584      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 252, 252, 128)     147584    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 126, 126, 128)     0         
_________________________________________________________________
dropout (Dropout)            (None, 126, 126, 128)     0         
_________________________________________________________________
flatten (Flatten)            (None, 2032128)           0         
_________________________________________________________________
dense (Dense)                (None, 128)               260112512 
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0